# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122997e+02     1.619023e+00
 * time: 0.08878493309020996
     1     1.105490e+01     9.079850e-01
 * time: 0.8614771366119385
     2    -1.213588e+01     1.114836e+00
 * time: 0.9755511283874512
     3    -3.402366e+01     8.217549e-01
 * time: 1.135667085647583
     4    -4.741145e+01     6.096374e-01
 * time: 1.2780511379241943
     5    -5.695579e+01     2.266537e-01
 * time: 1.418313980102539
     6    -5.982156e+01     1.365675e-01
 * time: 1.5294160842895508
     7    -6.088263e+01     6.293082e-02
 * time: 1.6374340057373047
     8    -6.129770e+01     6.164982e-02
 * time: 1.7485649585723877
     9    -6.160748e+01     4.021358e-02
 * time: 1.8592350482940674
    10    -6.185615e+01     2.987059e-02
 * time: 1.97064208984375
    11    -6.203629e+01     1.893770e-02
 * time: 2.075101137161255
    12    -6.209908e+01     1.539312e-02
 * time: 2.184998035430908
    13    -6.215991e+01     1.236581e-02
 * time: 2.295746088027954

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557661
    AtomicNonlocal      14.8522640
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485363 
    Xc                  -19.3336817

    total               -62.261666461008
